# Marketing Assistant

Copyright 2025, Denis Rothman

**Goal:** This notebook serves as the practical runtime for the **Strategic Marketing Engine**, as architected in Chapter 9. It demonstrates how the generic, multi-domain "Glass Box" Context Engine can be seamlessly repurposed to solve a variety of real-world marketing challenges without any changes to its core code.

This notebook will:
* Connect to the Pinecone knowledge base populated by the `Data_Ingestion_Marketing.ipynb` script.
* Use the three generic Control Deck templates to interact with the engine.
* Execute seven distinct marketing use cases, from enforcing brand voice to drafting a complete email nurture sequence.




# I. Inititalization

## GitHub

In [ ]:
# --- Downloading the utilities and the agents ---

# The !curl command is a simple and effective way to download raw files from a public GitHub repo.
# The -L flag ensures that it follows any redirects.

# print("Downloading helper files from public repository...")
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/helpers.py --output helpers.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/agents.py --output agents.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/registry.py.py --output registry.py.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/engine.py --output engine.py

# print("✅ Files downloaded successfully!")

In [ ]:
import requests
import os
from google.colab import userdata

def download_private_github_file(filename, destination_path="."):
    """
    Downloads a file from a private GitHub repository using a token
    stored in Colab Secrets.

    Requires a secret named 'GITHUB_TOKEN' to be set in the Colab UI.
    """
    # --- Configuration: Replace with your repository details ---
    owner = "Denis2054"
    repo = "Context-Engineering"
    branch = "main"
    # ---------------------------------------------------------

    try:
        # Securely fetch the GitHub token from Colab Secrets
        token = userdata.get('GITHUB_TOKEN')
        if not token:
            raise userdata.SecretNotFoundError("Secret 'GITHUB_TOKEN' not found.")
    except userdata.SecretNotFoundError:
        print("🛑 Error: Secret 'GITHUB_TOKEN' not found.")
        print("Please add your GitHub Personal Access Token to the Colab Secrets Manager.")
        return
    except Exception as e:
        print(f"An error occurred while accessing secrets: {e}")
        return

    # Construct the GitHub API URL for the file
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{filename}?ref={branch}"

    # Prepare headers for authentication and to request the raw file content
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/vnd.github.v3.raw"
    }

    try:
        # Make the request to the GitHub API
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # This will raise an HTTPError for bad responses (4xx or 5xx)

        # Determine the local filename
        local_filename = os.path.join(destination_path, os.path.basename(filename))

        # Save the file content locally
        with open(local_filename, "wb") as f:
            f.write(response.content)

        print(f"✅ Successfully downloaded '{filename}' to '{local_filename}'")

    except requests.exceptions.HTTPError as e:
        print(f"🛑 Error downloading '{filename}': {e}")
        if e.response.status_code == 404:
            print("   Please check that the owner, repo, file path, and branch are correct.")
        elif e.response.status_code == 401:
            print("   Authentication failed. Please check if your GITHUB_TOKEN is valid and has access to the repo.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# --- Example Usage: Download your utility files ---
# Ensure you have set 'GITHUB_TOKEN' in your Colab Secrets before running this.

download_private_github_file("commons/ch8/utils.py")
download_private_github_file("commons/ch8/helpers.py")
download_private_github_file("commons/ch8/agents.py")
download_private_github_file("commons/ch8/registry.py")
download_private_github_file("commons/ch8/engine.py")

## Installation and client setup

In [ ]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

## Context Engine library Import

In [ ]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [ ]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

# In Legal_Compliance_Assistant.ipynb (The "Engine Room" cell)

def execute_and_display(goal, config, client, pc, moderation_active=False):
    """
    Runs the context engine, now with an optional, two-stage moderation check.
    """
    # --- PRE-FLIGHT MODERATION CHECK (on user input) ---
    if moderation_active:
        print("--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=goal, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Goal failed pre-flight moderation. Execution halted.")
            return # Halt execution before calling the engine

    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config
    )

    # --- POST-FLIGHT MODERATION CHECK (on AI output) ---
    if result and moderation_active:
        print("\n--- [Safety Guardrail] Performing Post-Flight Moderation Check on Output ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=result, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Generated output failed post-flight moderation and will be redacted.")
            result = "[Content flagged as potentially harmful by moderation policy and has been redacted.]"

    # 2. Display the Final Result
    print("\n--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace
    print("\\n\\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # --- THIS LINE IS TO CREATE THE 'pp' OBJECT ---
        pp = pprint.PrettyPrinter(indent=2)

        pp.pprint(trace.steps)


## Control Deck configuration

In [ ]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#III.CONTROL DECKS

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [ ]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.
goal = "Summarize the key points of the Non-Disclosure Agreement."

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

In [ ]:
#@title CONTROL DECK TEMPLATE 1: High-Fidelity RAG

# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

# === CONTROL DECK 1: High-Fidelity RAG in a Legal Context ===
goal = "What are the key confidentiality obligations in the Service Agreement v1, and what is the termination notice period? Please cite your sources."

# === CONTROL DECK 1 (LIMIT TEST): Sanitization of Legal Testimony ===
#goal = "What did Mr. Smith advise his client regarding the assets?"

# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)

In [ ]:
#@title #CONTROL DECK 2: Brand Voice Enforcement

# 1. Define the Goal: A multi-step task that involves summarizing a large
#    document and then using that summary for a different purpose.
#    - DOMAIN: Any field with large documents (legal, scientific, corporate).
#    - KEY CAPABILITY: Tests the `Summarizer` agent and the engine's ability
#      to perform Context Chaining between the `Summarizer` and the `Writer`.
# goal = "[INSERT YOUR CONTEXT REDUCTION GOAL HERE]"

# === STEP 1: Define the Raw Text to be Refined ===

# This variable simulates the raw, unpolished output we might get from
# a simple query to the Researcher agent.
large_text_from_researcher = """
Check out our new SSD, the QuantumDrive Q-1. It's super fast with speeds up to 7,500 MB/s and it won't overheat thanks to its cool graphene heat spreader. You should totally buy it.
"""

print("✅ Raw text defined and ready for refinement.")

# === STEP 2: CONTROL DECK 2 - USE Case 1 Execute Brand Voice Enforcement ===

# 1. Define a goal that tells the engine to rewrite the text using the brand guide.
# Notice how we use the variable from the cell above as our input.
goal = f"""
First, summarize the key points of the official Brand Voice and Tone Guide.
Then, using that summary as a strict set of rules, rewrite the following rough draft to be perfectly on-brand.

--- TEXT TO REWRITE ---
{large_text_from_researcher}
"""

# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)

In [ ]:
#@title CONTROL DECK TEMPLATE 3: Grounded Reasoning & Hallucination Prevention

# 1. Define the Goal: A creative or factual task that is deliberately
#    outside the scope of the documents in the knowledge base.
#    - DOMAIN: Universal test applicable to any curated knowledge base.
#    - KEY CAPABILITY: Tests the `Researcher` agent's ability to report a
#      negative finding and the `Writer` agent's ability to handle it gracefully,
#      preventing hallucination.
# goal = "[INSERT YOUR OUT-OF-SCOPE GOAL HERE]"


# === CONTROL DECK 3: Grounded Reasoning and Hallucination Prevention ===
goal = "Write a persuasive opening statement for a trial involving a monkey that can fly a rocket."

# === CONTROL DECK 3 (LIMIT TEST): The Ambiguous Request ===
#goal = "Analyze the attached NDA and draft a pleading based on its terms."

# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)